In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join("..", "barrels"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pathlib import Path
from typing import List, Tuple, Dict

import cv2
import dataclass_array as dca
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import pandas as pd
import pycolmap
import scipy.io
from scipy.spatial import KDTree
from sklearn import linear_model
import transforms3d as t3d
import visu3d as v3d

import colmap_util as cutil

In [ ]:
# image_dir = Path("../dive-data/Dive8/clips/barrelddt1")
# image_dir = Path("../dive-data/Dive8/clips/barrel1")
image_dir = Path("../data/dive-data/Dive8/clips/barrel2")
# image_dir = Path("../dive-data/Dive8/clips/barrel3")
# image_dir = Path("../dive-data/Dive8/clips/barrelddt3")
# image_dir = Path("../dive-data/Dive8/clips/barreltriplet")

output_path = Path(f"../results/{image_dir.name}-reconstr")
img_navs = pd.read_csv(image_dir / "frame-time-nav.csv")
img_navs["timestamp"] = pd.to_datetime(img_navs["timestamp"])
reconstruction = pycolmap.Reconstruction(output_path / "colmap-out/mvs/sparse")
print(reconstruction.summary())

# Process and segment dense point cloud output

In [ ]:
cam2img = {}
for idx, img in reconstruction.images.items():
    cam2img[img.camera_id] = img

imgid2mask = {}
for idx, img in reconstruction.images.items():
    imgname = Path(img.name).stem
    maskpath = output_path / f"masks/{imgname}_mask_1.png"
    if maskpath.exists():
        imgid2mask[idx] = cv2.imread(str(maskpath), cv2.IMREAD_GRAYSCALE)

In [ ]:
reconstruction.images

In [ ]:
plypc = o3d.io.read_point_cloud(str(output_path / "openmvs-out/scene_dense.ply"))
plypts = np.asarray(plypc.points)
plycols = (np.asarray(plypc.colors) * 255).astype(np.uint8)

In [ ]:
print(list(reconstruction.images.values())[0].summary())

In [ ]:
print(list(reconstruction.cameras.values())[0].summary())

In [ ]:
H, W = (875, 1920)
pts, cols = cutil.get_pc(reconstruction)

ptcloud = v3d.Point3d(p=pts, rgb=cols)
ptcloud_dense = v3d.Point3d(p=plypts, rgb=plycols)
allmats = []
v3dcams: List[v3d.Camera] = []
camid2v3dcam = {}
for idx, img in reconstruction.images.items():
    cam = reconstruction.cameras[img.camera_id]
    foc_len = cam.focal_length
    cam_spec = v3d.PinholeCamera.from_focal(
        # resolution=(cam.height, cam.width),
        resolution=(H, W),
        focal_in_px=foc_len,
    )
    pmat = img.cam_from_world.matrix()
    # R = img.cam_from_world.rotation.matrix()
    # p = img.cam_from_world.translation
    # T = np.eye(4)
    # T[:3, :3] = R
    # T[:3, 3] = p
    # T = np.linalg.inv(np.vstack([pmat, [0, 0, 0, 1]]))
    T = np.vstack([pmat, [0, 0, 0, 1]])
    T = np.linalg.inv(T)
    # T[:3, :3] = -T[:3, :3].T
    v3dcam = v3d.Camera(
        spec=cam_spec,
        world_from_cam=v3d.Transform.from_matrix(T)
    )
    camid2v3dcam[img.camera_id] = v3dcam
    v3dcams.append(v3dcam)
allmats = np.array(allmats)
v3dcams = dca.stack(v3dcams)

In [ ]:
v3d.make_fig([
    ptcloud,
    v3dcams
])

In [ ]:
camrender = v3dcams[0].render(ptcloud_dense)
fig, ax = plt.subplots()
ax.imshow(camrender)
plt.show()

In [ ]:
# pc_to_seg = ptcloud
pc_to_seg = ptcloud_dense
npts = pc_to_seg.shape[0]
idxs = np.arange(npts)
barrelscores = np.zeros(npts)
floorscores = np.zeros(npts)
for imgid, mask in imgid2mask.items():
    img = reconstruction.images[imgid]
    v3dcam = camid2v3dcam[img.camera_id]
    pxpts = v3dcam.px_from_world @ pc_to_seg
    uvs = pxpts.p
    valid = (uvs[:, 0] >= 0) & (uvs[:, 0] <= W) & (uvs[:, 1] >= 0) & (uvs[:, 1] <= H)
    barrelmask = mask[uvs[valid].astype(int).T[1], uvs[valid].astype(int).T[0]] > 0
    barrelidxs = idxs[valid][barrelmask]
    flooridxs = idxs[valid][~barrelmask]
    barrelscores[barrelidxs] += 1
    floorscores[flooridxs] += 1
barrelyes = barrelscores > len(imgid2mask) / 3
segcols = np.zeros_like(pc_to_seg.rgb)
segcols[barrelyes] = [50, 222, 100]
segcols[~barrelyes] = [255, 0, 0]
segpc = v3d.Point3d(p=pc_to_seg.p, rgb=segcols)

In [ ]:
v3d.make_fig(segpc)

In [ ]:
x, y, z = pc_to_seg.p[:, 0], pc_to_seg.p[:, 1], pc_to_seg.p[:, 2]
matdata = {
    "scenex": x,
    "sceney": y,
    "scenez": z,
    "isbarrel": barrelyes
}
scipy.io.savemat(output_path / f"{image_dir.name}-dense-seg.mat", matdata)

# camera positions from nav data

In [ ]:
def haversine(lat1, lat2, lon1, lon2):
    """
    Calculates the haversine distance between two points.
    """
    R = 6378.137  # Radius of earth in KM
    dLat = lat2 * np.pi / 180 - lat1 * np.pi / 180
    dLon = lon2 * np.pi / 180 - lon1 * np.pi / 180
    a = np.sin(dLat / 2) * np.sin(dLat / 2) + np.cos(lat1 * np.pi / 180) * np.cos(
        lat2 * np.pi / 180
    ) * np.sin(dLon / 2) * np.sin(dLon / 2)
    c = 2 * np.arctan2(a ** (1 / 2), (1 - a) ** (1 / 2))
    d = R * c
    return d * 1000  # meters

def get_img_navrow(filename):
    img_navrows = img_navs[img_navs["filename"] == filename]
    return img_navrows.iloc[0]

def random_cylinder(x1, x2, r, npoints):
    # generates uniformly distributed points within the volume of
    # a defined cylinder
    x1 = np.array(x1)
    x2 = np.array(x2)
    axis = x2 - x1
    h = np.linalg.norm(axis)
    axis = axis / h
    axnull = scipy.linalg.null_space(np.array([axis]))
    axnull1 = axnull[:, 0]
    axnull2 = axnull[:, 1]
    # sqrt radius to get uniform distribution
    rand_r = r * np.sqrt(np.random.random(npoints))
    rand_theta = np.random.random(npoints) * 2 * np.pi
    rand_h = np.random.random(npoints) * h
    
    cosval = np.tile(axnull1, (npoints, 1)) * rand_r[..., None] * np.cos(rand_theta)[..., None]
    sinval = np.tile(axnull2, (npoints, 1)) * rand_r[..., None] * np.sin(rand_theta)[..., None]
    xyzs = cosval + sinval + np.tile(x1, (npoints, 1)) + rand_h[..., None] * np.tile(axis, (npoints, 1))
    return xyzs

In [ ]:
img_navs.head()

In [ ]:
origin_row = img_navs.iloc[0]
origin_x = origin_row["longitude"]
origin_y = origin_row["latitude"]
filename2transform = {}
all_transforms = []
for idx, row in img_navs.iterrows():
    navpos = row
    x = navpos["longitude"]
    y = navpos["latitude"]
    xdiff = haversine(origin_y, origin_y, origin_x, x)
    ydiff = haversine(origin_y, y, origin_x, origin_x)
    if x < origin_x:
        xdiff *= -1
    if y < origin_y:
        ydiff *= -1
    x = xdiff
    y = ydiff
    z = -navpos["depth"]
    yaw = navpos["yaw"]
    # yaw = navpos["heading"] * np.pi / 180
    pitch = navpos["pitch"]
    roll = navpos["roll"]
    reg2opt = np.array([
        [0, -1, 0],
        [0, 0, -1],
        [1, 0, 0],
    ])
    # pitch was written assuming regular frame, oops
    # roll is ambiguous, but is very small anyway, probably doesn't affect anything
    R0 = t3d.euler.euler2mat(0, 0, yaw - np.pi/2, axes="sxyz")
    R1 = t3d.euler.euler2mat(-pitch - (20 * np.pi / 180), 0, roll, axes="rxyz")
    R = R0 @ R1
    # R = t3d.euler.euler2mat(roll, pitch, yaw, axes="sxyz")
    # R = t3d.euler.euler2mat(yaw, pitch, roll, axes="rzyx")
    # R = t3d.euler.euler2mat(yaw, pitch, roll, axes="ryxz")
    # R = reg2opt @ R
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = [x, y, z]
    all_transforms.append(T)
    filename2transform[row["filename"]] = T
all_transforms = np.array(all_transforms)
all_transforms[:, 2, 3] = all_transforms[:, 2, 3] - np.mean(all_transforms[:, 2, 3])

cam_spec = v3d.PinholeCamera.from_focal(
    resolution=(H, W),
    focal_in_px=1360,
)
v3dnavcams = v3d.Camera(
    spec=cam_spec,
    world_from_cam=v3d.Transform.from_matrix(all_transforms)
)

In [ ]:
# R = t3d.euler.euler2mat(0, 0, 0, axes="rzyx")
R = t3d.euler.euler2mat(0, 0, np.pi/4, axes="sxyz") @ t3d.euler.euler2mat(np.pi/2, 0, np.pi/8, axes="rxyz")
T = np.eye(4)
T[:3, :3] = R
v3d.Camera(
    spec=cam_spec,
    world_from_cam=v3d.Transform.from_matrix(T)
).fig

In [ ]:
v3dcams.fig

In [ ]:
cyl = random_cylinder([4, 0, -2], [4, 0, 1], 1, 1000000)

rgb = np.zeros((cyl.shape[0], 3))
rgb[:, 0] = ((cyl[:, 2] - np.min(cyl[:, 2])) / (np.max(cyl[:, 2]) - np.min(cyl[:, 2]))) * 255
cylpt3d = v3d.Point3d(p=cyl, rgb=rgb)
v3d.make_fig(
    cylpt3d,
    v3dnavcams
)

In [ ]:
foundpairs = set()
realdists = []
sfmdists = []
ratiodists = []
for imgid, img in reconstruction.images.items():
    for imgid1, img1 in reconstruction.images.items():
        if imgid != imgid1:
            if (imgid, imgid1) in foundpairs or (imgid1, imgid) in foundpairs:
                continue
            v3dcam = camid2v3dcam[img.camera_id]
            v3dcam1 = camid2v3dcam[img1.camera_id]
            sfmdist = np.linalg.norm(v3dcam.world_from_cam.t - v3dcam1.world_from_cam.t)
            navrow = get_img_navrow(img.name)
            navrow1 = get_img_navrow(img1.name)
            horizdist = haversine(navrow["latitude"], navrow1["latitude"], navrow["longitude"], navrow1["longitude"])
            depthdist = np.abs(navrow["depth"] - navrow1["depth"])
            realdist = np.sqrt(horizdist ** 2 + depthdist ** 2)
            # T = filename2transform[img.name]
            # T1 = filename2transform[img1.name]
            # realdist = np.linalg.norm(T[:3, 3] - T1[:3, 3])
            realdists.append(realdist)
            sfmdists.append(sfmdist)
            ratiodists.append(realdist / sfmdist)
            foundpairs.add((imgid, imgid1))

# X = np.array([sfmdists, np.ones_like(sfmdists)]).T
X = np.array([sfmdists]).T
y = np.array(realdists)

In [ ]:
ransac = linear_model.RANSACRegressor(
    estimator=linear_model.LinearRegression(),
    max_trials=100,
    min_samples=0.8,
)
lr = linear_model.LinearRegression()
lr.fit(X, y)
ransac.fit(X, y)

inlier_mask = ransac.inlier_mask_
outlier_mask = ~inlier_mask

line_X = np.arange(X.min(), X.max())[:, np.newaxis]
line_y = lr.predict(line_X)
line_y_ransac = ransac.predict(line_X)

print("Estimated coefficients (linear regression, RANSAC):")
print("linreg coefficient:", lr.coef_, "ransac coefficient:", ransac.estimator_.coef_)

lw = 2
plt.scatter(
    X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers"
)
plt.scatter(
    X[outlier_mask], y[outlier_mask], color="gold", marker=".", label="Outliers"
)
plt.plot(line_X, line_y, color="navy", linewidth=lw, label="Linear regressor")
plt.plot(
    line_X,
    line_y_ransac,
    color="cornflowerblue",
    linewidth=lw,
    label="RANSAC regressor",
)
plt.legend(loc="lower right")
plt.xlabel("SFM distance")
plt.ylabel("Real distance (m)")
plt.show()

In [ ]:
measured = 0.8188
lr.coef_[0] * measured , ransac.estimator_.coef_[0] * measured